In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

Q 10종 분류 데이터를 통한 모델 학습 후 성능이 90%정도 나올 수 있도록 학습시켜주시오

CUDA 사용 가능 여부 확인

In [30]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

데이터 로드 및 전처리

In [41]:
batch_size=64
train_data=datasets.CIFAR10(root='./.data/',train=True,download=True,transform=
                              transforms.Compose([
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomCrop(32, padding=4),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ]))
test_data=datasets.CIFAR10(root='./.data/',train=False,download=True,transform=
                              transforms.Compose([
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomCrop(32, padding=4),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ]))
train_l = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_l = torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


모델 정의

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [43]:
model = Net().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

학습 함수 정의

In [44]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

평가 함수 정의

In [45]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

학습 및 평가

In [46]:
epochs = 30
for epoch in range(1, epochs + 1):
    train(model, train_l, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_l)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.299679
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1.929210
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.774862
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.323171
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.530523
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.326720
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.356515
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.309573
Test set: Average loss: 1.2277, Accuracy: 55.93%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.110108
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.028196
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.310566
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.955870
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.316832
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.019746
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.103830
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.163748
Test set: Average loss: 1.0006, Accuracy: 64.27%
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.020044
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.034788
Train Epoch: 3 [1280

성능 평가

In [37]:
if test_accuracy >= 90.0:
    print("모델의 성능이 90% 이상입니다.")
else:
    print("모델의 성능이 90% 미만입니다.")

모델의 성능이 90% 미만입니다.
